In [ ]:
import numpy as np

In [ ]:
# To avoid issues with escape characters etc
# we also read the sample from a file
with open("./13-sample.txt", "r") as FILE:
    data = FILE.read()

In [ ]:
with open("./13-input.txt", "r") as FILE:
    data = FILE.read()

In [ ]:
def read_track(data):
    """ We read the track model and remove the cars so we have a clean matrix """
    data = data.replace(">","-")
    data = data.replace("<","-")
    data = data.replace("^","|")
    data = data.replace("v","|")
    
    return np.matrix([list(line) for line in data.splitlines()])

track =read_track(data)
track.shape

In [ ]:
def track_to_string(track):
    """ Create a string representation of the track """
    string = []
    for line in track:
        string.append("".join(line.tolist()[0]))
        
    return "\n".join(string)

def plot_map(track, carts):
    """ Create a string representation of the track including the carts """
    track = track.copy()
    for cart in carts:
        track[cart.pos()[0],cart.pos()[1]] = cart.heading()
    return track_to_string(track)  


In [ ]:
# These define the rules for how carts behave at corners
# heading + corner => new direction
corner_rules = {
    ">\\": "v",
    "<\\": "^",
    "^\\": "<",
    "v\\": ">",
    ">/": "^",
    "</": "v",
    "^/": ">",
    "v/": "<"        
}

# defines the different directions and their order
rot_l = "v>^<v"
    
class Cart(object):

    def __init__(self, r, c, heading, track):
        self.__r = r
        self.__c = c
        self.__heading = heading
        self.__track = track
        self.__turns = 0

    def __str__(self):
        """ String representation shows a small part of track with cart at centre """
        return self.show_on_map(pad=2)

    def __repr__(self):
        return self.__str__()
    
    def drive(self):
        """ Moves the Cart along the track. """
        if self.__heading == ">":
            self.__c += 1
        elif self.__heading == "<":
            self.__c -= 1
        elif self.__heading == "v":
            self.__r += 1
        elif self.__heading == "^":
            self.__r -= 1
            
        # Now check if we change direction
        position = self.__track[self.__r,self.__c]
        
        if position == "+":
            if self.__turns % 3 == 0:
                self.__heading = rot_l[rot_l.index(self.__heading)+1]
            elif self.__turns % 3 == 2:
                self.__heading = rot_l[rot_l.rfind(self.__heading)-1]
            self.__turns+=1
        elif position in "\\/": 
            self.__heading = corner_rules[self.__heading + position]
            
    def pos(self):
        """ Return the position for this cart """
        return self.__r, self.__c

    def heading(self, heading=None):
        """ Set or return the heading for this cart """
        if heading is None:
            return self.__heading
        else:
            self.__heading = heading

    def show_on_map(self, pad=None):
        segment = self.__track.copy()
        segment[self.__r, self.__c] = self.__heading
        
        if pad is not None:        
            # Make sure we don't plot outside boundaries
            if self.__r < pad: plot_r = pad
            elif self.__r > self.__track.shape[0] - pad: plot_r = self.__track.shape[0] - pad
            else: plot_r = self.__r

            if self.__c < pad: plot_c = pad 
            elif self.__c > self.__track.shape[1] - pad: plot_c = self.__track.shape[1] - pad
            else: plot_c = self.__c

            segment = segment[plot_r-pad:plot_r+pad+1, plot_c-pad:plot_c+pad+1]
            
        return track_to_string(segment)        
            
# Read all the carts
carts = []
for r, line in enumerate(data.splitlines()):
    for c, char in enumerate(line):
        if char in "<>^v":
            carts.append(Cart(r,c,char,track))
        
for cart in carts:
    print(cart.pos())

# print(plot_map(track, carts))

In [ ]:
running = True
while running:
    carts.sort(key=lambda c: c.pos())
    for cart in carts:
        cart.drive()
        # Check for colisions
        for other in carts:
            if cart != other and cart.pos() == other.pos():
                print("CRASH!", cart.pos()[1], cart.pos()[0])
                running = False;
        if not running: break

# Part 2

We only need to tweak the routine slightly to track and remove dead carts

In [ ]:
# Read all the carts again
carts = []
for r, line in enumerate(data.splitlines()):
    for c, char in enumerate(line):
        if char in "<>^v":
            cart = Cart(r,c,char,track)
            cart.dead = False
            carts.append(cart)

In [ ]:
running = True
tick = -1
while running:
    tick += 1
    carts.sort(key=lambda c: c.pos())
    for cart in carts:
        if not cart.dead:
            cart.drive()

        # Check for colisions
        for other in carts:
            if cart != other and not other.dead and cart.pos() == other.pos():
                cart.dead = True
                other.dead = True
                print("CRASH!", tick, cart.pos()[1], cart.pos()[0])

    carts = [c for c in carts if not c.dead] 
    if len(carts) == 1:
        cart = carts[0]
        print("Last cart standing", cart.pos()[1], cart.pos()[0])
        break
